### Stable Diffusion v1.5

This is a legacy model from Stable Diffusion that can run on a Raspberry Pi 5 without further hacking

In [ ]:
# run this cell before trying prompts with Stable Diffusion v1.5

from diffusers import StableDiffusionPipeline
import torch
import lib

# define and initialize model, configure to run on CPU
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to("cpu")

In [ ]:
# describe your desired image here, and set number of denoising steps (higher improves quality, but takes linearly longer)
# around 30s / Iteration on a Raspberry Pi 5
prompt = "a cat"
filename = "image1.png"
inference_steps = 10

stream = lib.DiffuserStream(inference_steps)
image = pipe(
    prompt,
    num_inference_steps = inference_steps,
    callback_on_step_end = stream.write
    ).images[0]  

lib.ensure_dest_folder("output")
image.save(f"output/{filename}")

### SDXL-Turbo (Stable Diffusion XL)
This is a way larger model than SD v1.5 from above. In order to make it work on a Raspberry Pi 5, check out the repo's tutorial on `dphys-swapfile` within the setup-guide for the Raspberry Pi.

In [ ]:
# run this cell before trying prompts with Stable Diffusion XL Turbo

from diffusers import AutoPipelineForText2Image
import torch
import lib

model_id = "stabilityai/sdxl-turbo"
pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to("cpu")

In [ ]:
# describe your desired image here, and set number of denoising steps (higher improves quality, but takes linearly longer)
# around 2m 30s / step on a Raspberry Pi 5
prompt = "a dog"
filename = "image1.png"
inference_steps = 20

stream = lib.DiffuserStream(inference_steps, init_step=1)
image = pipe(
    prompt,
    num_inference_steps = inference_steps,
    callback_on_step_end = stream.write
).images[0]  
    
lib.ensure_dest_folder("output")
image.save(f"output/{filename}")

### DeepSeek Janus (does not work on a Raspberry Pi)
there was no success yet, to configure it for CPU-only and also it needs further dependencies. Visit the official [Doc](https://github.com/deepseek-ai/Janus?tab=readme-ov-file#janus)

In [ ]:
import os
import PIL.Image
import torch
import numpy as np
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor


# specify the path to the model
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "A stunning princess from kabul in red, white traditional clothing, blue eyes, brown hair",
    },
    {"role": "Assistant", "content": ""},
]

sft_format = vl_chat_processor.apply_sft_template_for_multi_turn_prompts(
    conversations=conversation,
    sft_format=vl_chat_processor.sft_format,
    system_prompt="",
)
prompt = sft_format + vl_chat_processor.image_start_tag


@torch.inference_mode()
def generate(
    mmgpt: MultiModalityCausalLM,
    vl_chat_processor: VLChatProcessor,
    prompt: str,
    temperature: float = 1,
    parallel_size: int = 16,
    cfg_weight: float = 5,
    image_token_num_per_image: int = 576,
    img_size: int = 384,
    patch_size: int = 16,
):
    input_ids = vl_chat_processor.tokenizer.encode(prompt)
    input_ids = torch.LongTensor(input_ids)

    tokens = torch.zeros((parallel_size*2, len(input_ids)), dtype=torch.int).cuda()
    for i in range(parallel_size*2):
        tokens[i, :] = input_ids
        if i % 2 != 0:
            tokens[i, 1:-1] = vl_chat_processor.pad_id

    inputs_embeds = mmgpt.language_model.get_input_embeddings()(tokens)

    generated_tokens = torch.zeros((parallel_size, image_token_num_per_image), dtype=torch.int).cuda()

    for i in range(image_token_num_per_image):
        outputs = mmgpt.language_model.model(inputs_embeds=inputs_embeds, use_cache=True, past_key_values=outputs.past_key_values if i != 0 else None)
        hidden_states = outputs.last_hidden_state
        
        logits = mmgpt.gen_head(hidden_states[:, -1, :])
        logit_cond = logits[0::2, :]
        logit_uncond = logits[1::2, :]
        
        logits = logit_uncond + cfg_weight * (logit_cond-logit_uncond)
        probs = torch.softmax(logits / temperature, dim=-1)

        next_token = torch.multinomial(probs, num_samples=1)
        generated_tokens[:, i] = next_token.squeeze(dim=-1)

        next_token = torch.cat([next_token.unsqueeze(dim=1), next_token.unsqueeze(dim=1)], dim=1).view(-1)
        img_embeds = mmgpt.prepare_gen_img_embeds(next_token)
        inputs_embeds = img_embeds.unsqueeze(dim=1)


    dec = mmgpt.gen_vision_model.decode_code(generated_tokens.to(dtype=torch.int), shape=[parallel_size, 8, img_size//patch_size, img_size//patch_size])
    dec = dec.to(torch.float32).cpu().numpy().transpose(0, 2, 3, 1)

    dec = np.clip((dec + 1) / 2 * 255, 0, 255)

    visual_img = np.zeros((parallel_size, img_size, img_size, 3), dtype=np.uint8)
    visual_img[:, :, :] = dec

    os.makedirs('generated_samples', exist_ok=True)
    for i in range(parallel_size):
        save_path = os.path.join('generated_samples', "img_{}.jpg".format(i))
        PIL.Image.fromarray(visual_img[i]).save(save_path)


generate(
    vl_gpt,
    vl_chat_processor,
    prompt,
)